# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 60787.01it/s]


In [5]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [6]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [7]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [8]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[-0.0872, -0.1837,  0.0663,  ...,  0.8223, -0.9755, -1.6482],
         [ 1.3381, -0.0301,  0.9966,  ...,  0.1041,  1.3338,  0.3834],
         [-0.0453, -0.3511, -1.3705,  ...,  0.1287, -0.0406, -0.7384],
         ...,
         [ 1.5341, -0.5658,  2.5760,  ...,  0.7668, -0.1331,  0.3681],
         [ 1.5341, -0.5658,  2.5760,  ...,  0.7668, -0.1331,  0.3681],
         [ 1.5341, -0.5658,  2.5760,  ...,  0.7668, -0.1331,  0.3681]],

        [[-0.7317, -0.6956, -0.3393,  ...,  0.2617,  0.4112,  1.0699],
         [-1.3148,  0.3945,  0.4143,  ...,  0.1664,  0.1821,  1.4388],
         [ 1.3474, -1.1592, -1.2798,  ..., -1.7441, -1.1508, -0.2235],
         ...,
         [ 1.5341, -0.5658,  2.5760,  ...,  0.7668, -0.1331,  0.3681],
         [ 1.5341, -0.5658,  2.5760,  ...,  0.7668, -0.1331,  0.3681],
         [ 1.5341, -0.5658,  2.5760,  ...,  0.7668, -0.1331,  0.3681]],

        [[ 0.7209, -0.6180,  0.4204,  ..., -0.5001,  1.5654, -0.5699],
         [-0.2522, -0.5939,  1.1027,  ..., -1

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [9]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [10]:
w_0 = nn.Linear(d_model, d_model)

In [11]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [12]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [13]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [14]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0479, 0.0324, 0.0634,  ..., 0.0532, 0.0532, 0.0532],
          [0.0255, 0.0495, 0.0866,  ..., 0.0334, 0.0334, 0.0334],
          [0.0432, 0.0314, 0.0736,  ..., 0.0565, 0.0565, 0.0565],
          ...,
          [0.0279, 0.0610, 0.0734,  ..., 0.0376, 0.0376, 0.0376],
          [0.0279, 0.0610, 0.0734,  ..., 0.0376, 0.0376, 0.0376],
          [0.0279, 0.0610, 0.0734,  ..., 0.0376, 0.0376, 0.0376]],

         [[0.0579, 0.0379, 0.0388,  ..., 0.0512, 0.0512, 0.0512],
          [0.0386, 0.0420, 0.0434,  ..., 0.0562, 0.0562, 0.0562],
          [0.0514, 0.0505, 0.0615,  ..., 0.0619, 0.0619, 0.0619],
          ...,
          [0.0661, 0.0207, 0.0630,  ..., 0.0719, 0.0719, 0.0719],
          [0.0661, 0.0207, 0.0630,  ..., 0.0719, 0.0719, 0.0719],
          [0.0661, 0.0207, 0.0630,  ..., 0.0719, 0.0719, 0.0719]],

         [[0.0352, 0.0536, 0.0844,  ..., 0.0333, 0.0333, 0.0333],
          [0.0407, 0.0493, 0.0892,  ..., 0.0477, 0.0477, 0.0477],
          [0.0278, 0.0740, 0.0298,  ..., 0

In [15]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [16]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [17]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 1.5717e-01, -3.1369e-01, -5.9587e-02,  ...,  3.9965e-02,
          -4.1008e-02, -2.3296e-01],
         [ 1.0249e-01, -2.2069e-01, -4.2081e-02,  ...,  1.3163e-02,
          -2.9182e-02, -2.5673e-01],
         [ 4.8828e-02, -2.7699e-01, -2.4381e-02,  ..., -5.3123e-02,
          -1.4591e-02, -2.0055e-01],
         ...,
         [ 7.6182e-02, -3.0714e-01, -1.0364e-02,  ...,  2.2050e-02,
           2.0028e-02, -2.4123e-01],
         [ 7.6182e-02, -3.0714e-01, -1.0364e-02,  ...,  2.2050e-02,
           2.0028e-02, -2.4123e-01],
         [ 7.6182e-02, -3.0714e-01, -1.0364e-02,  ...,  2.2050e-02,
           2.0028e-02, -2.4123e-01]],

        [[ 4.7528e-01, -9.8734e-02, -1.9330e-01,  ...,  1.0549e-02,
           1.8985e-02, -3.3002e-01],
         [ 4.8786e-01, -1.0205e-01, -1.7467e-01,  ...,  5.6268e-02,
          -3.6668e-03, -2.8469e-01],
         [ 5.0575e-01, -7.6409e-02, -1.5484e-01,  ...,  3.6863e-02,
          -5.6308e-02, -3.6636e-01],
         ...,
         [ 4.8573e-01, -1

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [18]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = self.w_q(q)  # (B, L, d_model)
    k = self.w_k(k)  # (B, L, d_model)
    v = self.w_v(v)  # (B, L, d_model)
    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [19]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [20]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[-0.0110,  0.0470, -0.0620,  ..., -0.0875,  0.0084, -0.1096],
         [ 0.0024,  0.0604, -0.0475,  ..., -0.0565,  0.0504, -0.1013],
         [-0.0150, -0.0011, -0.0347,  ..., -0.0224,  0.0982, -0.1170],
         ...,
         [-0.0142,  0.1204, -0.0975,  ..., -0.1024, -0.0124, -0.1269],
         [-0.0142,  0.1204, -0.0975,  ..., -0.1024, -0.0124, -0.1269],
         [-0.0142,  0.1204, -0.0975,  ..., -0.1024, -0.0124, -0.1269]],

        [[-0.2620, -0.0029, -0.0328,  ..., -0.2944, -0.1437, -0.1228],
         [-0.2605, -0.1386, -0.0489,  ..., -0.2830, -0.1722, -0.0812],
         [-0.2897, -0.0571, -0.0280,  ..., -0.3137, -0.1660, -0.1217],
         ...,
         [-0.3023, -0.0305, -0.0695,  ..., -0.3105, -0.2095, -0.1495],
         [-0.3023, -0.0305, -0.0695,  ..., -0.3105, -0.2095, -0.1495],
         [-0.3023, -0.0305, -0.0695,  ..., -0.3105, -0.2095, -0.1495]],

        [[-0.1329, -0.0812, -0.1102,  ..., -0.2355, -0.2508, -0.1708],
         [-0.0920, -0.0041, -0.0873,  ..., -0